<a href="https://www.kaggle.com/code/ocanaydin/financial-sentiment-bert?scriptVersionId=123626002" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# DATA PREPROCESSING

In [ ]:
#Fetch data from dataset.
df = pd.read_csv("/kaggle/input/financial-sentiment-analysis/data.csv")
df.head(5)

In [ ]:
print(f"Negative : {sum(df['Sentiment'] == 'negative')}")
print(f"Neutral : {sum(df['Sentiment'] == 'neutral')}")
print(f"Positive : {sum(df['Sentiment'] == 'positive')}")

In [ ]:
#Check if any null value in dataset.
print(df.isnull().sum())
#We see that there is no null value so no need to drop any row.
print(len(df))

In [ ]:
#Some libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt

In [ ]:
#Apply One hot encoder to convert sentiment column to every seperate column(negative,neutral,positive.) 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

encoded_y = encoder.fit_transform(df["Sentiment"])
dummy_y = tf.keras.utils.to_categorical(encoded_y)

In [ ]:
#Convert array to dataframe and concatenate two dataframes.
df_sentiment = pd.DataFrame(dummy_y,columns = encoder.classes_,index = df["Sentence"].index)

In [ ]:
#Drop sentiment column to change it to its one hot encoded version.
df.drop(columns = ["Sentiment"],inplace = True)

In [ ]:
df = pd.concat([df,df_sentiment],axis = 1)
df.head(5)

In [ ]:
#Split dataset as train and test.
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df["Sentence"],df.iloc[:,1:],test_size = 0.5,
                                                 random_state = 42,shuffle = False)

In [ ]:
#Create validation dataset from test dataset.
X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size = 0.5,random_state = 42,shuffle = False)

# CHOOSE A BERT MODEL TO FINE TUNE

In [ ]:
#You can pick your BERT MODEL from tensorflow hub. 
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/2"
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
#Define preprocess model to tokenize words with respect to BERT.
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
#See example about tokenizer.Notice => Bert model takes input as list.
idx = X_train.index[10]
ex = [X_train[idx]]
ex_preprocessed = bert_preprocess_model(ex)
print(f"Sentence : {ex}")
print(f'Keys       : {list(ex_preprocessed.keys())}')
print(f'Shape      : {ex_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {ex_preprocessed["input_word_ids"][0, :20]}')
print(f'Input Mask : {ex_preprocessed["input_mask"][0, :20]}')
print(f'Type Ids   : {ex_preprocessed["input_type_ids"][0, :20]}')

**DEFINE YOUR MODEL**

In [ ]:
def build_classifier_model():
    #Get text input and preprocess it.
    text_input = tf.keras.layers.Input(shape = (),dtype = tf.string,name = "text")
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess,name = "preprocessing")
    encoder_inputs = preprocessing_layer(text_input)
    #Use encoder to apply BERT MODEL.
    encoder = hub.KerasLayer(tfhub_handle_encoder,trainable = True,name = "BERT_encoder")
    outputs = encoder(encoder_inputs)
    #You can think of a pooled_output as a embedding for the entire movie_review.
    net = outputs["pooled_output"]
    #Droupout and output layers.
    net = tf.keras.layers.Dropout(0.25)(net)
    net = tf.keras.layers.Dense(3,activation = "softmax",name = "classifier")(net)
    return tf.keras.Model(text_input,net)
    

In [ ]:
#Assign classifer model see the model on graph.
model = build_classifier_model()
tf.keras.utils.plot_model(model)

# MODEL TRAINING

In [ ]:
#Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
#Compile the model
model.compile(optimizer = optimizer,loss = "categorical_crossentropy",metrics = "accuracy") 

In [ ]:
print(f"Training model : {tfhub_handle_encoder}")
history = model.fit(x = X_train,y = y_train,validation_data = (X_val,y_val),epochs = 15,batch_size = 128)

In [ ]:
loss,accuracy = model.evaluate(X_test,y_test)
print(f"Loss : {loss} , acc : {accuracy}")

# PLOT HISTORY

In [ ]:
history = history 
#Metrics
acc = history["accuracy"]
loss = history["loss"]
val_acc = history["val_accuracy"]
val_loss = history["val_loss"]

epochs = range(1,len(acc) + 1)
#Plot Accuracy
fig = plt.figure(figsize = (10,6))
plt.subplot(2,1,1)
plt.plot(epochs,acc,"r",label = "Training acc")
plt.plot(epochs,val_acc,"b",label = "Validation acc")
plt.ylabel("Accuracy")
plt.legend()
#Plot Loss
plt.subplot(2,1,2)
plt.plot(epochs,loss,"r",label = "Training loss")
plt.plot(epochs,val_loss,"b",label = "Validation loss")
plt.ylabel("Loss")
plt.legend()

plt.show()

# SAVE MODEL

In [ ]:
model.save("financial_sentiment_BERT")

In [ ]:
model_saved = tf.saved_model.load("financial_sentiment_BERT")

# PREDICTION

In [ ]:
#(1)Get text example from test dataset.
n1 = 20
n2 = 30

test_ex = X_test[n1:n2]
preds = model.predict([test_ex])
#Find biggest element in array to predict class.
preds = np.argmax(preds,axis = 1) 
classes = encoder.classes_

for i in range(len(preds)):
    real = np.argmax(y_test[n1 + i : n1+ 1 + i])
    print(f"Prediction : {classes[preds[i]]} Real : {classes[real]}")

In [ ]:
#These sentences are test sentences which are predicted above by the model.
X_test[n1 : n2]

In [ ]:
#(2) You can write your own sentence and predict its sentiment.
own_sentence = ["The indicators tell me that this is not a good investment."]
own_pred = model.predict(own_sentence)

print(own_pred)
pred_index = np.argmax(own_pred,axis = 1)
print(classes[pred_index])